In [1]:
#| default_exp core

In [2]:
#| export
import kagglehub
    
class Model:
    def __init__(self):
        self.svg_default = '<svg width="100" height="100" viewBox="0 0 100 100"><circle cx="50" cy="50" r="40" fill="red" /></svg>'
        pass
    def predict(self, prompt: str) -> str:
        return self.svg_default